In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU ", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전:  2.1.0
즉시 실행 모드:  True
허브 버전:  0.7.0
GPU  사용 불가능


In [17]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [20]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.',
       b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends

In [22]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

In [23]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.209591  , -2.7093675 ,  3.6802928 , -1.0291991 , -4.1671185 ,
        -2.4566064 , -2.2519937 , -0.36589956,  1.9485804 , -3.1104462 ,
        -2.4610963 ,  1.3139242 , -0.9161584 , -0.16625322, -3.723651  ,
         1.8498232 ,  3.499562  , -1.2373022 , -2.8403084 , -1.213074  ],
       [ 1.9055302 , -4.11395   ,  3.6038654 ,  0.28555924, -4.658998  ,
        -5.5433393 , -3.2735848 ,  1.9235417 ,  3.8461034 ,  1.5882455 ,
        -2.64167   ,  0.76057523, -0.14820506,  0.9115291 , -6.45758   ,
         2.3990374 ,  5.0985413 , -3.2776263 , -3.2652326 , -1.2345369 ],
       [ 3.6510668 , -4.7066135 ,  4.71003   , -1.7002777 , -3.7708545 ,
        -3.709126  , -4.222776  ,  1.946586  ,  6.1182513 , -2.7392752 ,
        -5.4384456 ,  2.7078724 , -2.1263676 , -0.7084146 , -5.893995  ,
         3.1602864 ,  3.8389287 , -3.318196  , -5.1542974 , -2.4051712 ]],
      dtype=float32)>

In [24]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
history = model.fit(train_data.shuffle(1000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================].0157 - accuracy: 0.59 - 2s 993ms/step - loss: 3.7712 - accuracy: 0.598 - 2s 685ms/step - loss: 3.8625 - accuracy: 0.588 - 2s 539ms/step - loss: 3.7962 - accuracy: 0.589 - 2s 449ms/step - loss: 3.8259 - accuracy: 0.586 - 2s 391ms/step - loss: 3.7529 - accuracy: 0.593 - 2s 350ms/step - loss: 3.7578 - accuracy: 0.592 - 3s 317ms/step - loss: 3.7615 - accuracy: 0.596 - 3s 293ms/step - loss: 3.7578 - accuracy: 0.596 - 3s 273ms/step - loss: 3.7231 - accuracy: 0.596 - 3s 258ms/step - loss: 3.7105 - accuracy: 0.592 - 3s 245ms/step - loss: 3.6652 - accuracy: 0.595 - 3s 234ms/step - loss: 3.6443 - accuracy: 0.598 - 3s 224ms/step - loss: 3.5890 - accuracy: 0.599 - 3s 216ms/step - loss: 3.6091 - accuracy: 0.597 - 3s 208ms/step - loss: 3.5674 - accuracy: 0.599 - 3s 203ms/step - loss: 3.5583 - accuracy: 0.598 - 4s 198ms/step - loss: 3.5688 - accuracy: 0.597 - 4s 197ms/step - loss: 3.5590 - accuracy: 0.597 - 4s 194ms/step - loss: 3.5794 - accuracy: 0.5

30/30 [==============================] - ETA: 6s - loss: 2.0538 - accuracy: 0.69 - ETA: 4s - loss: 2.2626 - accuracy: 0.65 - ETA: 3s - loss: 2.5168 - accuracy: 0.64 - ETA: 3s - loss: 2.4882 - accuracy: 0.64 - ETA: 2s - loss: 2.4753 - accuracy: 0.64 - ETA: 2s - loss: 2.4685 - accuracy: 0.65 - ETA: 2s - loss: 2.4171 - accuracy: 0.65 - ETA: 2s - loss: 2.4312 - accuracy: 0.65 - ETA: 2s - loss: 2.3970 - accuracy: 0.65 - ETA: 2s - loss: 2.3622 - accuracy: 0.65 - ETA: 2s - loss: 2.3547 - accuracy: 0.65 - ETA: 2s - loss: 2.3521 - accuracy: 0.65 - ETA: 2s - loss: 2.3436 - accuracy: 0.65 - ETA: 1s - loss: 2.3252 - accuracy: 0.65 - ETA: 1s - loss: 2.3156 - accuracy: 0.65 - ETA: 1s - loss: 2.3040 - accuracy: 0.65 - ETA: 1s - loss: 2.3128 - accuracy: 0.65 - ETA: 1s - loss: 2.3155 - accuracy: 0.65 - ETA: 1s - loss: 2.3253 - accuracy: 0.65 - ETA: 1s - loss: 2.3234 - accuracy: 0.65 - ETA: 1s - loss: 2.3202 - accuracy: 0.65 - ETA: 0s - loss: 2.3110 - accuracy: 0.65 - ETA: 0s - loss: 2.3118 - accuracy: 

30/30 [==============================] - ETA: 7s - loss: 3.0513 - accuracy: 0.61 - ETA: 5s - loss: 3.2582 - accuracy: 0.60 - ETA: 4s - loss: 3.3339 - accuracy: 0.60 - ETA: 3s - loss: 3.2202 - accuracy: 0.60 - ETA: 3s - loss: 3.1301 - accuracy: 0.60 - ETA: 3s - loss: 2.9393 - accuracy: 0.61 - ETA: 2s - loss: 2.8014 - accuracy: 0.62 - ETA: 2s - loss: 2.6404 - accuracy: 0.62 - ETA: 2s - loss: 2.5468 - accuracy: 0.63 - ETA: 2s - loss: 2.4132 - accuracy: 0.64 - ETA: 2s - loss: 2.3177 - accuracy: 0.65 - ETA: 2s - loss: 2.2572 - accuracy: 0.65 - ETA: 1s - loss: 2.2062 - accuracy: 0.65 - ETA: 1s - loss: 2.1322 - accuracy: 0.66 - ETA: 1s - loss: 2.0819 - accuracy: 0.66 - ETA: 1s - loss: 2.0422 - accuracy: 0.67 - ETA: 1s - loss: 2.0021 - accuracy: 0.67 - ETA: 1s - loss: 1.9703 - accuracy: 0.67 - ETA: 1s - loss: 1.9429 - accuracy: 0.67 - ETA: 1s - loss: 1.9249 - accuracy: 0.67 - ETA: 1s - loss: 1.8886 - accuracy: 0.67 - ETA: 0s - loss: 1.8581 - accuracy: 0.67 - ETA: 0s - loss: 1.8350 - accuracy: 

30/30 [==============================] - ETA: 7s - loss: 0.8982 - accuracy: 0.76 - ETA: 4s - loss: 0.8128 - accuracy: 0.76 - ETA: 3s - loss: 0.8996 - accuracy: 0.75 - ETA: 3s - loss: 0.9541 - accuracy: 0.74 - ETA: 3s - loss: 0.9693 - accuracy: 0.75 - ETA: 2s - loss: 0.9507 - accuracy: 0.75 - ETA: 2s - loss: 0.9729 - accuracy: 0.75 - ETA: 2s - loss: 0.9585 - accuracy: 0.75 - ETA: 2s - loss: 0.9553 - accuracy: 0.75 - ETA: 2s - loss: 0.9575 - accuracy: 0.75 - ETA: 2s - loss: 0.9594 - accuracy: 0.75 - ETA: 1s - loss: 0.9632 - accuracy: 0.75 - ETA: 1s - loss: 0.9708 - accuracy: 0.75 - ETA: 1s - loss: 0.9718 - accuracy: 0.75 - ETA: 1s - loss: 0.9649 - accuracy: 0.75 - ETA: 1s - loss: 0.9753 - accuracy: 0.75 - ETA: 1s - loss: 0.9709 - accuracy: 0.75 - ETA: 1s - loss: 0.9776 - accuracy: 0.75 - ETA: 1s - loss: 0.9721 - accuracy: 0.75 - ETA: 1s - loss: 0.9862 - accuracy: 0.75 - ETA: 0s - loss: 0.9781 - accuracy: 0.75 - ETA: 0s - loss: 0.9840 - accuracy: 0.75 - ETA: 0s - loss: 0.9744 - accuracy: 

In [28]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 1.037
accuracy: 0.713
